In [1]:
from twitter import *
import requests
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from matplotlib.path import Path

In [23]:
class TwitterCustom(object):
    def __init__(self):
        consumer_key = 'LFTXnbNVhCtLV2T2dG0yA'
        consumer_secret = '2DX6L81tUHP8Clxz8FvuXcaMvHTAXuUkXbzz7y6U'
        access_token = '77452634-avWq03PwpORlOtKoa4ctg082fTdFlBbKT0ihC8XZw'
        access_token_secret = 'cnopinzlsPRi6zBOMGdAVf0EuMM97eCQF5UPoyTyY'

        self.api = Twitter(auth=OAuth(access_token, access_token_secret, consumer_key, consumer_secret))
        
    def get_tweet(self, id):
        return self.api.statuses.show(_id=724694725601669120)

In [84]:
def get_geoname_data(geoname):
    return requests.get(u'http://api.geonames.org/postalCodeSearchJSON?placename={0}&maxRows=100&username=demo&style=FULL'.format(geoname)).json()

In [85]:
polygone = {
    'point': [(0., 0., 5.), (1., 0., 5.), (1., 1., 5.), (0., 1., 5.)]
}

polygone2 = {
    'point': [(0.5, 0.5, 5.), (0.75, 0.5, 5.), (0.75, 0.75, 5.), (0.5, 0.75, 5.)]
}

In [86]:
def make_graph(polygones):
    fig = plt.figure()
    ax = Axes3D(fig)
    
    x = []
    y = []
    z = []
    for poly in polygones:
        for point in poly['point']:
            x.append(point[0])
            y.append(point[1])
            z_temp = point[2]
            
            for poly2 in polygones:
                if not poly2 == poly:
                    if Path([[p[0], p[1]] for p in poly2['point']]).contains_point([point[0], point[1]]):
                        z_temp += poly2['point'][0][2]
            z.append(z_temp)
        
    ax.plot_trisurf(x, y, z)
        
    ax.set_xlim3d(min(x), max(x))
    ax.set_ylim3d(min(y), max(y))
    ax.set_zlim3d(min(z), max(z))
    plt.show()

In [87]:
make_graph([polygone, polygone2])

In [74]:
def determinate_tweet_localisation(tweet):
    # Get area by geoname in tweet
    
    # Get area by timezone
    
    # Get area by user localisation in profile
    
    # Get area by user bio
    a = ""

In [75]:
def get_tweet_localisation(tweet_id):
    t = Twitter()
    return determinate_tweet_localisation(t.get_tweet(tweet_id))

In [24]:
t = TwitterCustom()
tweet = t.get_tweet('724694725601669120')

In [25]:
tweet

{'contributors': None,
 'coordinates': None,
 'created_at': 'Mon Apr 25 20:21:01 +0000 2016',
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/lkd3N2szkA',
    'expanded_url': 'http://twitter.com/cyprien_g/status/724694725601669120/photo/1',
    'id': 724694698489729025,
    'id_str': '724694698489729025',
    'indices': [32, 55],
    'media_url': 'http://pbs.twimg.com/media/Cg6h_RoW0AEMtZh.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/Cg6h_RoW0AEMtZh.jpg',
    'sizes': {'large': {'h': 1820, 'resize': 'fit', 'w': 1024},
     'medium': {'h': 1067, 'resize': 'fit', 'w': 600},
     'small': {'h': 604, 'resize': 'fit', 'w': 340},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'type': 'photo',
    'url': 'https://t.co/lkd3N2szkA'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'extended_entities': {'media': [{'display_url': 'pic.twitter.com/lkd3N2szkA',
    'expanded_url': 'http://twitter.com/cyprien_g/status/724694725601669120/ph